In [1]:
 # Dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import pandas as pd

In [2]:
csv_file_path = ("../Car price data/accord_data.csv")

In [3]:
# Using Splinter Scraping data
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
# URL of page to be scraped
url = 'https://losangeles.craigslist.org/search/cto?min_price=2000&auto_make_model=accord&min_auto_year=2008&max_auto_year=2018&min_auto_miles=0&max_auto_miles=1000000&condition=10&condition=20&condition=30&condition=40'

In [ ]:
# Scrape the data based on post info
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html,"html.parser")
next_page = soup.find("a",class_ = "button next")["href"]

titles = []
prices = []
links = []
car_data = []

while next_page:
    print(" - - - - - - - - - - - Scraping New Page Starts!! - - - - - - - - - ")
    html = browser.html
    soup = BeautifulSoup(html,"html.parser")
    results = soup.find_all('li', class_="result-row")
    
    
    
    
    for result in results:
    # Error handling
        try:
            # Identify and return title of listing
            title = result.find('a', class_="result-title").text
            # Identify and return price of listing
            price = result.a.span.text
            # Identify and return link to listing
            link = result.a['href']

            # Print results only if title, price, and link are available
            if (title and price and link):
                titles.append(title)
                prices.append(price)
                links.append(link)
                
                post_link = result.a["href"]
                response2 = requests.get(post_link)
                soup2 = BeautifulSoup(response2.text,"html.parser")
                attributes = soup2.find_all("p",class_="attrgroup")
                attribute_list = attributes[1].find_all("span")
                attr_dic = {}
                
                try:
                
                    tag = attributes[0].find("span").find("b").text.split(" ")
                
                    yearlist = [str(num) for num in list(range(1900, 2020))]
                    for word in tag:
                        if word in yearlist:
                            year = word
                            for attr in attribute_list:
                                try:
                                    key,value = attr.text.split(":")
                                    attr_dic[key] = value
                                    attr_dic["Year"] = year
                                except ValueError:
                                    pass
                            if word not in tag:
                                attr_dic["Year"] = "NA"
                    
                    
                except:
                    attr_dic["Year"] = "NA"
                
    
                car_data.append(attr_dic)
                
                
        except AttributeError as e:
            print(e)
    try:
        browser.click_link_by_partial_text('next')
        next_page = soup.find("a",class_ = "button next")["href"]
    except:
        print("- - - - - - - - - This is the last page we can find!!!- - - - - -")
        next_page = False

 - - - - - - - - - - - Scraping New Page Starts!! - - - - - - - - - 
'NoneType' object has no attribute 'text'


In [ ]:
# Store info as dataframe
info_df = pd.DataFrame({
    "Info" : titles,
    "Price" : prices,
    "links" : links
})

In [ ]:
# Show full dataframe
pd.set_option('display.max_colwidth', -1)
# Merge two dataframe
car_df = pd.DataFrame(car_data)
car_df["Title"] = info_df["Info"]
car_df["Price"] = info_df["Price"]
car_df["Link"] = info_df["links"]
car_df["Make"] = "Honda"
car_df["Model"] = "Accord"
car_df = car_df[['Title','Make', 'Model','Year', 'Condition', 'Cylinders', 'Drive', 'Fuel', 'Odometer',
       'Paint_color', 'Size', 'Title_status', 'Transmission', 'Type', 
       'Price', 'VIN', 'Link']]
car_df.head(50)

In [ ]:
# Out put as CSV file
car_df.to_csv(csv_file_path)